In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import mean_absolute_error
import dataframe_image as dfi
import hopsworks
import joblib
import os

In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/4247


Connected. Call `.close()` to terminate connection gracefully.


In [15]:
electricity_data_fg = fs.get_feature_group(name="new_electricity_data_fg", version=1)
display(electricity_data_fg.show(5))

2023-01-13 21:29:31,617 INFO: USE `id2223_lab1_featurestore`
2023-01-13 21:29:32,125 INFO: SELECT `fg0`.`hydro_water_reservoir` `hydro_water_reservoir`, `fg0`.`nuclear` `nuclear`, `fg0`.`other` `other`, `fg0`.`solar` `solar`, `fg0`.`wind_onshore` `wind_onshore`, `fg0`.`datetime` `datetime`, `fg0`.`day_ahead_price` `day_ahead_price`, `fg0`.`total_load` `total_load`, `fg0`.`temperature` `temperature`
FROM `id2223_lab1_featurestore`.`new_electricity_data_fg_1` `fg0`


,hydro_water_reservoir,nuclear,other,solar,wind_onshore,datetime,day_ahead_price,total_load,temperature
0,1238.0,5401.0,228.0,242.0,151.0,1661515200000,628.10,8442.0,22.9
1,314.0,6336.0,195.0,0.0,1116.0,1651960800000,1.64,6673.0,7.4
2,370.0,4630.0,526.0,135.0,1034.0,1641812400000,54.93,8204.0,2.0
3,1154.0,6992.0,909.0,66.0,451.0,1647154800000,75.63,10840.0,5.0
4,362.0,5699.0,257.0,103.0,849.0,1662566400000,3.18,7351.0,17.0


In [ ]:
## Get model 

mr = project.get_model_registry()
model = mr.get_model("SE3_elec_price_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/electricity_price.pkl")

In [18]:
# Get the dataset, sort to get the latest 24 rows
full_dataset = electricity_data_fg.select_all().read()  
full_dataset = full_dataset.sort_values(by=['datetime'])
display(full_dataset.tail(24))

2023-01-13 21:32:38,795 INFO: USE `id2223_lab1_featurestore`
2023-01-13 21:32:39,336 INFO: SELECT `fg0`.`hydro_water_reservoir` `hydro_water_reservoir`, `fg0`.`nuclear` `nuclear`, `fg0`.`other` `other`, `fg0`.`solar` `solar`, `fg0`.`wind_onshore` `wind_onshore`, `fg0`.`datetime` `datetime`, `fg0`.`day_ahead_price` `day_ahead_price`, `fg0`.`total_load` `total_load`, `fg0`.`temperature` `temperature`
FROM `id2223_lab1_featurestore`.`new_electricity_data_fg_1` `fg0`


,hydro_water_reservoir,nuclear,other,solar,wind_onshore,datetime,day_ahead_price,total_load,temperature
8779,597.0,5798.0,557.0,0.0,869.0,1672614000000,57.91,9632.0,7.4
8800,578.0,5797.0,583.0,0.0,930.0,1672617600000,51.67,9491.0,7.5
8780,558.0,5796.0,578.0,0.0,926.0,1672621200000,52.86,9388.0,7.4
8790,553.0,5798.0,579.0,0.0,912.0,1672624800000,44.16,9290.0,7.7
8798,552.0,5796.0,585.0,0.0,871.0,1672628400000,50.08,9256.0,7.6
8796,597.0,5796.0,638.0,0.0,830.0,1672632000000,70.73,9402.0,7.5
8788,691.0,5796.0,745.0,0.0,797.0,1672635600000,102.56,10154.0,7.5
8794,840.0,5797.0,794.0,0.0,754.0,1672639200000,138.19,11148.0,7.1
8795,882.0,5795.0,861.0,0.0,638.0,1672642800000,145.98,11597.0,7.2
8789,911.0,5795.0,830.0,0.0,515.0,1672646400000,147.05,11628.0,6.8


In [22]:
predict_df = full_dataset.set_index("datetime").drop(columns=['day_ahead_price'])
predict_df.head(24)

,hydro_water_reservoir,nuclear,other,solar,wind_onshore,total_load,temperature
datetime,,,,,,,
1640991600000,824.0,5841.0,892.0,0.0,1062.0,9685.0,5.3
1640995200000,818.0,5842.0,851.0,0.0,999.0,9612.0,5.4
1640998800000,818.0,5839.0,858.0,0.0,958.0,9601.0,5.4
1641002400000,838.0,5839.0,859.0,0.0,888.0,9552.0,5.7
1641006000000,842.0,5840.0,856.0,0.0,788.0,9449.0,5.3
1641009600000,841.0,5839.0,880.0,0.0,734.0,9614.0,4.4
1641013200000,868.0,5839.0,913.0,0.0,646.0,9920.0,4.7
1641016800000,903.0,5840.0,987.0,0.0,564.0,10269.0,4.3
1641020400000,920.0,5840.0,1021.0,0.0,471.0,10534.0,2.2


In [ ]:
y_pred = model.predict(predict_df)
y_pred

In [24]:
prediction_date = full_dataset.iloc[:]['datetime']
prediction_date

1282    1640991600000
1384    1640995200000
7209    1640998800000
4637    1641002400000
6484    1641006000000
            ...      
8783    1672682400000
8793    1672686000000
8785    1672689600000
8784    1672693200000
8801    1672696800000
Name: datetime, Length: 8802, dtype: int64

In [ ]:
# Get actual price

# todo

In [ ]:
# DF for monitoring

# todo

In [ ]:
electricity_monitor_fg = fs.get_or_create_feature_group(name="electricity_predictions_fg",
                                            version=1,
                                            primary_key=["datetime"],
                                            description="Electricity price Prediction & Outcome Monitoring")

electricity_monitor_fg.insert(monitor_df, write_options={"wait_for_job": False})